In [1]:
import os
import sys
import math
import json

cquav_root_path = os.path.abspath('../')
if cquav_root_path not in sys.path:
    sys.path.append(cquav_root_path)

import numpy as np
import cadquery as cq
from jupyter_cadquery import show
import matplotlib.pyplot as plt
%matplotlib inline

from cquav.wing.airfoil import Airfoil, load_airfoils_collection
from cquav.wing.profile import AirfoilSection, ThreeChamberBoxedWingSection
from cquav.wing.rect_console import RectangularWingConsole
from cquav.materials import IsotropicMaterial, FluidProperties

Overwriting auto display for cadquery Workplane and Shape


## Load airfoils collection

In [2]:
airfoils_collection = load_airfoils_collection()
airfoil_data = airfoils_collection["NACA 4 digit airfoils"]["NACA 2412 (naca2412-il)"]
airfoil = Airfoil(airfoil_data)

## Define constants

In [3]:
velocity_max = 35 #[m/s]
air_density = 1.225 #[kg/m^3]
kinematic_viscosity = 1.460*1e-5 ## [m^2/s]
dyn_airpressure_max = 0.5 * air_density * velocity_max**2 ## dynamic pressure
delta_max = 0.1 ## relative wing tip displacement
payload_mass = 10 ## kg
g = 9.81 ## [m/s**2]
load_factor = 1.2
cd_payload = 0.5
payload_cross_section_area = math.pi*0.1**2 / 4

## Materials

In [4]:
XPS_foam = IsotropicMaterial(30, 1e3, 25*1e6)
# https://www.mdpi.com/2073-4360/12/1/47

Fiberglass_laminate = IsotropicMaterial(1800, 290*1e6, 12.4*1e9)
#https://laminatedplastics.com/fiberglasslaminates.pdf

Carbon_laminate = IsotropicMaterial(1500, 450*1e6, 35*1e9)
# https://www.researchgate.net/publication/259461841_Mechanical_Properties_of_Carbon_FiberEpoxy_Composites_Effects_of_Number_of_Plies_Fiber_Contents_and_Angle-Ply_Layers

In [5]:
air_props = FluidProperties(air_density, velocity_max, kinematic_viscosity)

## Build reinforced wing console

In [6]:
## Fixed parameters, mm
console_chord = 260
console_length = 900

max_chord_length = 1000
min_chord_length = 50

max_console_length = 12000
min_console_length = 50

MATERIALS = {"box": Fiberglass_laminate, "shell": Fiberglass_laminate, "foam": XPS_foam}

In [7]:
airfoil_section = AirfoilSection(airfoil, chord=console_chord)
wing_console = RectangularWingConsole(airfoil_section, length=console_length, materials=MATERIALS,
                                      min_length=min_console_length, max_length=max_console_length, 
                                      min_chord=min_chord_length, max_chord=max_chord_length)

number of profile data points: 35
number of brep control points: 12
new front wall position: 0.05


In [8]:
assy = cq.Assembly()
assy.add(wing_console.foam, name="foam", color=cq.Color("lightgray"))
assy.add(wing_console.front_box, name="left_box", color=cq.Color("yellow"))
assy.add(wing_console.central_box, name="central_box", color=cq.Color("yellow"))
assy.add(wing_console.rear_box, name="right_box", color=cq.Color("yellow"))
assy.add(wing_console.shell, name="shell", color=cq.Color("lightskyblue2"))
show(assy, angular_tolerance=0.1)

100% ⋮————————————————————————————————————————————————————————————⋮ (5/5)  0.68s


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

## Find solution of the required wing length that satisfies the max wing tip displacement requirement

In [9]:
airfoil_section = AirfoilSection(airfoil, chord=150)
reynolds = airfoil_section.eval_reynolds(air_props)
reynolds

number of profile data points: 35
number of brep control points: 12


359589.0410958904

In [10]:
box_section = ThreeChamberBoxedWingSection(airfoil_section) #thickness_tol=0
wing_console = RectangularWingConsole(box_section, materials=MATERIALS, 
                                      min_length=min_console_length, max_length=max_console_length, 
                                      min_chord=min_chord_length, max_chord=max_chord_length)

new front wall position: 0.05


In [11]:
alpha_min_drag = wing_console.airfoil_section.airfoil.alpha_min_drag(reynolds)
alpha_min_drag

0.3604633440024395

In [12]:
solved_console_length = wing_console.fit_length_to_required_tip_deflection(
    alpha_min_drag, air_props, load_factor=load_factor
)

Finding wing console length for the chord = 150
Console length: 4614.493834438756 [mm]
Сonsole mass: 4.895838300423805 [kg]
Console spect ratio: 30.763292229591706
Lift force: 161.24994435891736 [N]
Absolute tip deflection: 14686.385385230078 [mm]
Relative tip deflection: 318.26644291131294 %
Box thickness: 1.427, [mm]
Tip deflection error: 3.0826644291131293

Console length: 7435.506165561243 [mm]
Сonsole mass: 7.888847005707124 [kg]
Console spect ratio: 49.570041103741616
Lift force: 259.82805449407743 [N]
Absolute tip deflection: 99005.83535573386 [mm]
Relative tip deflection: 1331.5278496344404 %
Box thickness: 1.427, [mm]
Tip deflection error: 13.215278496344405

Console length: 2871.0123311224866 [mm]
Сonsole mass: 3.0460571918911272 [kg]
Console spect ratio: 19.140082207483243
Lift force: 100.32532174865797 [N]
Absolute tip deflection: 2200.690007055979 [mm]
Relative tip deflection: 76.65205694869204 %
Box thickness: 1.427, [mm]
Tip deflection error: 0.6665205694869204

Console 

In [13]:
solved_console_length

1457.6691355004075

In [14]:
wing_console.stats(alpha_min_drag, air_props, load_factor=load_factor)

Length: 1457.6691355004075, [mm]
Chord: 150, [mm]
Aspect ratio: 9.717794236669384
Area: 0.2186503703250611, [m^2]
----------
Mass: 1.5465428361878988, [kg] (box: 0.9566951728265329, foam: 0.027119495312970882, shell: 0.562728168048395)
Angle of attack: 0.3604633440024395, [degrees]
Excess lift force: 32.73014596820137, [N]
Drag force: 1.3201885631129033, [N]
Lift to weight ratio: 2.7983907782328705
Center of aerodynamic pressure (lift): (37.5, 728.8345677502037), [mm, mm]
----------
Reinforcement box thickness: 1.427, [mm]
Shell thickness: 1, [mm]
Bend stress: 23.811780864455574, [MPa]
Shear stress: 0.6084273817833253, [MPa]
Von Mises stress: 23.835088830830145, [MPa]
Safety factor: 12.166935984937115


## Find solution of the required wing console size that ensures sufficient lift force

In [15]:
required_console_lift_force = 0.5 * load_factor * payload_mass * g

In [16]:
solved_console_chord = wing_console.fit_chord_to_required_lift_force(
    alpha_min_drag, air_props, required_console_lift_force, load_factor=load_factor  #, method="direct" #, method="brentq"   # method="shgo"
)

number of profile data points: 35
number of brep control points: 12
new front wall position: 0.05
Finding wing console length for the chord = 412.86771068759987
Current chord: 412.86771068759987, [mm]
Console length: 6474.983245261212, [mm]
Console excess lift force: 94.84787859102369, [N]
Required lift force: 58.86, [N]
Console mass: 39.8988696103532, [kg]
Box thickness: 3.927, [mm]
Absolute error: 35.98787859102369, [N]

number of profile data points: 35
number of brep control points: 12
new front wall position: 0.05
Finding wing console length for the chord = 637.1322893124001
Current chord: 637.1322893124001, [mm]
Console length: 9342.09699158343, [mm]
Console excess lift force: -258.76474674413225, [N]
Required lift force: 58.86, [N]
Console mass: 128.83915785639277, [kg]
Box thickness: 6.06, [mm]
Absolute error: -317.62474674413227, [N]

number of profile data points: 35
number of brep control points: 12
new front wall position: 0.05
Finding wing console length for the chord = 27

In [17]:
wing_console.stats(alpha_min_drag, air_props, load_factor=load_factor)

Length: 2706.5806667280563, [mm]
Chord: 236.7938424984255, [mm]
Aspect ratio: 11.430114221597856
Area: 0.6409016361064869, [m^2]
----------
Mass: 6.186342904529429, [kg] (box: 4.425559927810692, foam: 0.12548791748818058, shell: 1.6352950592305566)
Angle of attack: 0.3604633440024395, [degrees]
Excess lift force: 58.859999951235, [N]
Drag force: 3.0095455970858587, [N]
Lift to weight ratio: 1.808508047113067
Center of aerodynamic pressure (lift): (59.198460624606376, 1353.2903333640281), [mm, mm]
----------
Reinforcement box thickness: 2.252, [mm]
Shell thickness: 1, [mm]
Bend stress: 20.21530697188744, [MPa]
Shear stress: 0.43917981919072735, [MPa]
Von Mises stress: 20.22961375578867, [MPa]
Safety factor: 14.335419524113108


In [18]:
assy = cq.Assembly()
assy.add(wing_console.foam, name="foam", color=cq.Color("lightgray"))
assy.add(wing_console.front_box, name="left_box", color=cq.Color("yellow"))
assy.add(wing_console.central_box, name="central_box", color=cq.Color("yellow"))
assy.add(wing_console.rear_box, name="right_box", color=cq.Color("yellow"))
assy.add(wing_console.shell, name="shell", color=cq.Color("lightskyblue2"))
show(assy, angular_tolerance=0.1)

100% ⋮————————————————————————————————————————————————————————————⋮ (5/5)  3.74s


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…